In [1]:
import os
import sys
import requests
import json
from inspect import getmembers, isfunction


#connexion a la db
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
from flask_sqlalchemy import sqlalchemy
from sqlalchemy import create_engine
from functions_db import *
from urls_tb_ddets import *
from urls_tb_dreets import *
from urls_tb_siae import *
from urls_tb_cd import *
from urls_tb_dgefp import *
 

#Import librairies annexes
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
from io import StringIO


#Filter all warnings 
import warnings
warnings.filterwarnings('ignore')

import time

## Connecting to the API

In [7]:
#Getting the previous month
start = time.time()
first = today.replace(day=1)
lastMonth = first - datetime.timedelta(days=1)
lastCoupleMonth = first + relativedelta(months=-2)
lastMonth_ = (lastMonth.strftime("%Y-%m"))
lastCoupleMonth_ = (lastCoupleMonth.strftime("%Y-%m"))
month = (lastMonth.strftime("%Y-%m"))


#Contrairement au TB publics je calcule les infos semaine par semaine (ou deux semaines d'un coup quand j'oublie de le lancer...)
date = '2022-11-14'
today = '2022-12-04'
print('It took', time.time()-start, 'seconds.')

It took 0.0 seconds.


In [8]:
#Ici j'ai créé un dico pour pouvoir boucler sur tous les départements

regions = {
    
"Auvergne-Rhone-Alpes":
    {
        "01---Ain": {""},
        "03---Allier": {""},
        "07---Ardeche": {""},
        "15---Cantal": {""},
        "26---Drome": {""},
        "38---Isere": {""},
        "42---Loire": {""},
        "43---Haute-Loire": {""},
        "63---Puy-de-Dome": {""},
        "69---Rhone": {""},
        "26---Drome": {""},
        "73---Savoie": {""},
        "74---Haute-Savoie": {""}
    },    

"Bourgogne-Franche-Comte":
    {
        """21---Cote-d-Or""": {""},
        "25---Doubs": {""},
        "39---Jura": {""},
        "58---Nievre": {""},
        "70---Haute-Saone": {""},
        "71---Saone-et-Loire": {""},
        "89---Yonne": {""},
        "90---Territoire-de-Belfort": {""}
    },  
    
"Bretagne":
    {
        """22---Cotes-d-Armor""": {""},
        "29---Finistere": {""},
        "35---Ille-et-Vilaine": {""},
        "56---Morbihan": {""}
    },
   
"Centre-Val-de-Loire":
    {
        """18---Cher""": {""},
        "28---Eure-et-Loire": {""},
        "36---Indre": {""},
        "37---Indre-et-Loire": {""},
        "41---Loir-et-Cher": {""},
        "45---Loiret": {""}
    },
   
"Corse":
    {
        "2A---Corse-du-Sud": {""},
        "2B---Haute-Corse": {""}
    },
    
"Grand-Est":
    {
        "08---Ardennes": {""},
        "10---Aube": {""},
        "51---Marne": {""},
        "52---Haute-Marne": {""},
        "54---Meurthe-et-Moselle": {""},
        "55---Meuse": {""},
        "57---Moselle": {""},
        "67---Bas-Rhin": {""},
        "68---Haut-Rhin": {""},
        "88---Vosges": {""}
    },
    
"Guadeloupe":
    {
        "971---Guadeloupe": {""}
    },  

"Guyane":
    {
        "973---Guyane": {""}
    }, 
    
"Mayotte":
    {
        "976---Mayotte": {""}
    }, 
    
"Hauts-de-France":
    {
        "02---Aisne": {""},
        "59---Nord": {""},
        "60---Oise": {""},
        "62---Pas-de-Calais": {""},
        "80---Somme": {""}
    },    

"Ile-de-France":
    {
        "75---Paris": {""},
        "77---Seine-et-Marne": {""},
        "78---Yvelines": {""},
        "91---Essonne": {""},
        "92---Hauts-de-Seine": {""},
        "93---Seine-Saint-Denis": {""},
        "94---Val-de-Marne": {""},
        """95---Val-d-Oise""": {""},
    },
 
"La-Reunion":
    {
        "974---La-Reunion": {""}
    }, 
    
"Martinique":
    {
        "972---Martinique": {""}
    },  
    
'Normandie': 
    {
        '14---Calvados':{""},
        '27---Eure':{""}, 
        '50---Manche':{""},
        '61---Orne':{""}, 
        '76---Seine-Maritime':{""}
    },
    
'Nouvelle-Aquitaine': 
    {
        '16---Charente':{""},
        '17---Charente-Maritime':{""},
        '19---Correze':{""},
        '23---Creuse':{""},
        '24---Dordogne':{""}, 
        '33---Gironde':{""},
        '40---Landes':{""},
        '47---Lot-et-Garonne':{""},
        '64---Pyrenees-Atlantiques':{""},
        '79---Deux-Sevres':{""},
        '86---Vienne':{""},
        '87---Haute-Vienne':{""}
    },

'Occitanie': 
    {
        '09---Ariege':{""},
        '11---Aude':{""},
        '12---Aveyron':{""},
        '30---Gard':{""},
        '31---Haute-Garonne':{""},
        '32---Gers':{""},
        '34---Herault':{""},
        '46---Lot':{""},
        '48---Lozere':{""},
        '65---Hautes-Pyrenees':{""},
        '66---Pyrenees-Orientales':{""},
        '81---Tarn':{""},
        '82---Tarn-et-Garonne':{""}
    },

'Pays-de-la-Loire': 
    {
        '44---Loire-Atlantique':{""},
        '49---Maine-et-Loire':{""},
        '53---Mayenne':{""},
        '72---Sarthe':{""},
        '85---Vendee':{""}
    },
    
'Provence-Alpes-Cote-d-Azur': 
    {'04---Alpes-de-Haute-Provence':{""},
     '05---Hautes-Alpes':{""},
     '06---Alpes-Maritimes':{""},
     '13---Bouches-du-Rhone':{""},
     '83---Var':{""},
     '84---Vaucluse':{""}
    }

}

In [9]:
#Ici j'ai fait un mélange un peu sioux Dico/liste pour pouvoir lancer mes requêtes (fonctions dans la cellule d'apres)

cd= {}
siae_hiring = {}
pe_conversion = {}
pe_state = {}
pe_delay = {}
pe_tension = {}
ddets_iae = {}
ddets_hiring = {}
dreets_iae = {}
dreets_hiring = {}
dr_pe_conversion = {}
dr_pe_delay = {}
dr_pe_state = {}
dr_pe_tension = {}
#J'ai compartimenté en différentes listes parce que j'ai eu l'impression que c'était moins long à faire tourner
list_tb_dpt = [cd,ddets_iae,ddets_hiring]
list_tb_siae = [siae_hiring]
list_tb_dpt_pe = [pe_tension,pe_delay,pe_state,pe_conversion]
list_tb_reg = [dreets_iae,dreets_hiring]
list_tb_reg_pe = [dr_pe_conversion,dr_pe_delay,dr_pe_state,dr_pe_tension]
token = "12345"

for i in regions.keys():
    for j in regions[i]:
        cd[j] = ("""https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fcd%25252F"""+i+"""%25252F"""+j+"""&token_auth="""+token)
        siae_hiring[j] = ("""https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fsiae%25252Fhiring%25252F"""+i+"""%25252F"""+j+"""&token_auth="""+token )
        pe_conversion[j] = ("https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fpe%25252Fconversion%25252Fmain%25252F"""+i+"""%25252F"""+j+"""&token_auth="""+token)
        pe_delay[j] = ("https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fpe%25252Fdelay%25252Fmain%25252F"""+i+"""%25252F"""+j+"""&token_auth="""+token)
        pe_state[j] = ("https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fpe%25252Fstate%25252Fmain%25252F"""+i+"""%25252F"""+j+"""&token_auth="""+token)
        pe_tension[j] = ("https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fpe%25252Ftension%25252F"""+i+"""%25252F"""+j+"""&token_auth="""+token)
        ddets_iae[j] = ("""https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fddets%25252Fiae%25252F"""+i+"""%25252F"""+j+"""&token_auth="""+token)
        ddets_hiring[j] = ("""https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fddets%25252Fhiring%25252F"""+i+"""%25252F"""+j+"""&token_auth="""+token)
        
for i in regions.keys():
    dreets_iae[i] = ("""https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fdreets%25252Fiae%25252F"""+i+"""&token_auth="""+token)
    dreets_hiring[i] = ("""https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fdreets%25252Fhiring%25252F"""+i+"""&token_auth="""+token)
    dr_pe_conversion[i] = ("https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fpe%25252Fconversion%25252Fmain%25252F"""+i+"""%25252Fdrpe&token_auth="""+token)
    dr_pe_delay[i] = ("https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fpe%25252Fdelay%25252Fmain%25252F"""+i+"""%25252Fdrpe&token_auth="""+token)
    dr_pe_state[i] = ("https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fpe%25252Fstate%25252Fmain%25252F"""+i+"""%25252Fdrpe&token_auth="""+token)
    dr_pe_tension[i] = ("https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fpe%25252Ftension%25252F"""+i+"""%25252Fdrpe&token_auth="""+token)
    
dgefp = """https://stats.data.gouv.fr/index.php?module=API&format=CSV&idSite=117&period=week&date="""+date+""","""+today+"""&method=API.get&filter_limit=100&format_metrics=1&expanded=1&translateColumnNames=1&language=fr&segment=pageUrl%3D%3Dhttps%25253A%25252F%25252Femplois.inclusion.beta.gouv.fr%25252Fstats%25252Fdgefp%25252Fiae%25252F&token_auth="""+token

In [10]:
#On récupère les infos lorsque les TBs sont présents à l'échelle départementale
def create_private_df_dpt(url):
    data = []
    for i in regions.keys():
        for j in regions[i]:
            s = requests.get(url[j]).text 
            df = pd.read_csv(StringIO(s))
            df["Département"] = j
            df["Tableau de bord"] = str(url[j][300:320])
            data.append(df)
    df_final = pd.concat(data)
    return(df_final)

#On récupère les infos lorsque les TBs sont présents à l'échelle régionale
def create_private_df_reg(url):
    data = []
    for i in regions.keys():
        s = requests.get(url[i]).text 
        df = pd.read_csv(StringIO(s))
        df["Région"] = i
        df["Tableau de bord"] = str(url[i][300:320])
        data.append(df)
    df_final = pd.concat(data)
    return(df_final)

### A partir d'ici je récupère les données en lançant mes fonctions pour chaque liste de TBs

In [ ]:
tb = []
start = time.time()
for i in list_tb_dpt:
    df_final = create_private_df_dpt(i)
    tb.append(df_final)
print('It took', time.time()-start, 'seconds.')

In [7]:
start = time.time()

df_final = create_private_df_dpt(siae_hiring)
tb.append(df_final)
print('It took', time.time()-start, 'seconds.')

It took 53.476627588272095 seconds.


In [14]:
start = time.time()
for i in list_tb_dpt_pe:
    df_final = create_private_df_dpt(i)
    tb.append(df_final)
print('It took', time.time()-start, 'seconds.')

It took 345.52719378471375 seconds.


In [15]:
start = time.time()
for i in list_tb_reg:
    df_final = create_private_df_reg(i)
    tb.append(df_final)
print('It took', time.time()-start, 'seconds.')

It took 30.211318731307983 seconds.


In [16]:
start = time.time()
for i in list_tb_reg_pe:
    df_final = create_private_df_reg(i)
    tb.append(df_final)
print('It took', time.time()-start, 'seconds.')

It took 63.219380378723145 seconds.


In [17]:
data = []
s = requests.get(dgefp).text 
df = pd.read_csv(StringIO(s))
df["Tableau de bord"] = "dgefp"
data.append(df)
df_final = pd.concat(data)
tb.append(df_final)

In [19]:
df = pd.concat(tb)


### Nettoyage des données

In [21]:
df = df[~df["Date"].isna()]

In [22]:
df["Tableau de bord"].replace(to_replace='[^a-z]', value=' ', regex=True,inplace=True)
df["Tableau de bord"].replace(to_replace='(?:cd.*)', value='cd', regex=True,inplace=True)

In [23]:
#Noms des TBs a reprendre

df["Tableau de bord"].replace(to_replace='pe       tension     ', value="tb 162 - Fiches de poste en tension PE",inplace=True)
df["Tableau de bord"].replace(to_replace='pe       tension    ', value="tb 162 - Fiches de poste en tension PE",inplace=True)
df["Tableau de bord"].replace(to_replace='pe       delay      ', value="tb 168 - Délai d'entrée en IAE",inplace=True)
df["Tableau de bord"].replace(to_replace='pe       state      ', value="tb 149 - Candidatures orientées PE",inplace=True)
df["Tableau de bord"].replace(to_replace='pe       conversion ', value="tb 169 - Taux de transformation PE",inplace=True)
df["Tableau de bord"].replace(to_replace='cd', value="tb 118 - Données IAE CD",inplace=True)
df["Tableau de bord"].replace(to_replace='siae       hiring   ', value="tb 165 - Recrutement SIAE",inplace=True)
df["Tableau de bord"].replace(to_replace='ddets       iae     ', value="tb 117 - Données IAE DREETS/DDETS",inplace=True)
df["Tableau de bord"].replace(to_replace='ddets       hiring  ', value="tb 160 - Facilitation de l'embauche DREETS/DDETS",inplace=True)
df["Tableau de bord"].replace(to_replace='dreets       iae    ', value="tb 117 - Données IAE DREETS/DDETS",inplace=True)
df["Tableau de bord"].replace(to_replace='dgefp', value="TB DGEFP",inplace=True)

In [24]:
df["Tableau de bord"].unique()

array(['tb 118 - Données IAE CD', 'tb 117 - Données IAE DREETS/DDETS',
       "tb 160 - Facilitation de l'embauche DREETS/DDETS",
       'tb 162 - Fiches de poste en tension PE',
       "tb 168 - Délai d'entrée en IAE",
       'tb 149 - Candidatures orientées PE',
       'tb 169 - Taux de transformation PE'], dtype=object)

In [25]:
df["Date"].replace(to_replace='(?:,.*)', value='', regex=True,inplace=True)
df["Date"] = df["Date"].apply(lambda x: str(x).strip())
df["Date"] = pd.to_datetime(df["Date"])

In [26]:
df["Nom Département"] = df["Département"]
df["Nom Département"].replace(to_replace='(?:---)', value='-', regex=True,inplace=True)
df["Département"].replace(to_replace='\D', value='', regex=True,inplace=True)
df['Département'] = df['Département'].apply(lambda x: str(x).strip())

### Très important ! Colonnes à récupérer.   
J'ai été assez exhaustif donc ça ne devrait pas poser de problèmes de récupèrer ça comme ça


In [27]:
df=df[['Date', 'Département', "Nom Département", 'Tableau de bord', 'Visiteurs uniques',
       'Visites', 'Actions',
       "Nombre maximum d'actions en une visite", 'Rebonds',
       'Temps total passé par les visiteurs (en secondes)', 'Visites retour',
       'Actions des visites retour', 'Visiteurs uniques de retour',
       'Visiteurs connus', 'Actions maximum dans une visite de retour',
       'Pourcentage de rebond pour les visites retour',
       "Nombre moyen d'actions par visiteur connu",
       'Durée moyenne des visites pour les visiteurs connus (en secondes)',
       'Temps moyen de connexion','Moy. heure du serveur', 
       'Conversions', 'Visites avec conversions', 'Revenu',
       'Taux de conversion', 'nb_conversions_returning_visit',
       'nb_visits_converted_returning_visit', 'revenue_returning_visit',
       'conversion_rate_returning_visit', 'Vues de page',
       'Vues de page uniques', 'Téléchargements', 'Téléchargements uniques',
       'Liens sortants', 'Liens sortants uniques', 'Recherches',
       'Taux de rebond', 'Actions par visite',
       'Durée moy. des visites (en secondes)',
       "Moy. Durée d'une nouvelle visite (en sec)",
       'Moy. Actions par nouvelle visite',
       'Taux de rebond pour une nouvelle visite', 'Nouvelle Visite',
       'Actions de Nouvelles Visites', 'Nouveaux visiteurs uniques',
       'Nouveaux Utilisateurs', 'max_actions_new', 'nb_conversions_new_visit',
       'nb_visits_converted_new_visit', 'revenue_new_visit',
       'conversion_rate_new_visit']]

In [29]:
df

,Date,Département,Nom Département,Tableau de bord,Visiteurs uniques,Visites,Actions,Nombre maximum d'actions en une visite,Rebonds,Temps total passé par les visiteurs (en secondes),...,Taux de rebond pour une nouvelle visite,Nouvelle Visite,Actions de Nouvelles Visites,Nouveaux visiteurs uniques,Nouveaux Utilisateurs,max_actions_new,nb_conversions_new_visit,nb_visits_converted_new_visit,revenue_new_visit,conversion_rate_new_visit
0,2022-11-07,03,03-Allier,tb 118 - Données IAE CD,1.0,1.0,11.0,11.0,0.0,397.0,...,0 %,1.0,11.0,1.0,0.0,11.0,1.0,1.0,0.0,100 %
0,2022-11-07,07,07-Ardeche,tb 118 - Données IAE CD,1.0,1.0,33.0,33.0,0.0,222.0,...,0 %,1.0,33.0,1.0,0.0,33.0,1.0,1.0,0.0,100 %
0,2022-11-07,39,39-Jura,tb 118 - Données IAE CD,1.0,1.0,24.0,24.0,0.0,184.0,...,0 %,1.0,24.0,1.0,0.0,24.0,1.0,1.0,0.0,100 %
0,2022-11-07,37,37-Indre-et-Loire,tb 118 - Données IAE CD,1.0,1.0,14.0,14.0,0.0,1112.0,...,0 %,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-11-07,02,02-Aisne,tb 118 - Données IAE CD,1.0,1.0,49.0,49.0,0.0,1236.0,...,0 %,1.0,49.0,1.0,0.0,49.0,1.0,1.0,0.0,100 %
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2022-11-07,nan,NaN,tb 149 - Candidatures orientées PE,2.0,2.0,28.0,17.0,0.0,5337.0,...,0 %,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-11-07,nan,NaN,tb 149 - Candidatures orientées PE,1.0,1.0,24.0,24.0,0.0,1121.0,...,0 %,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-11-07,nan,NaN,tb 149 - Candidatures orientées PE,1.0,1.0,17.0,17.0,0.0,368.0,...,0 %,1.0,17.0,1.0,0.0,17.0,0.0,0.0,0.0,0 %
0,2022-11-07,nan,NaN,tb 162 - Fiches de poste en tension PE,1.0,1.0,12.0,12.0,0.0,1106.0,...,0 %,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
